In [108]:
using Plots
using ParticleFilters
using Distributions
using StaticArrays
using LinearAlgebra
using Random
using StatsBase
using Reel
using POMDPs
using POMDPSimulators
using POMDPPolicies
using POMDPModelTools
using GridInterpolations
using RLInterface

┌ Info: Precompiling RLInterface [3065fe53-3878-5aed-aae7-ab641be43734]
└ @ Base loading.jl:1242
┌ Warning: Module POMDPs with build ID 1501028945801 is missing from the cache.
│ This may mean POMDPs [a93abf59-7444-517b-a68a-c42f96afdd7d] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1000
┌ Info: Skipping precompilation since __precompile__(false). Importing RLInterface [3065fe53-3878-5aed-aae7-ab641be43734].
└ @ Base loading.jl:1017


In [343]:
include("atan2.jl")
include("obs.jl")
include("polargrid.jl")
rng = MersenneTwister(2);


In [126]:
# random walk

POS_0 = [6.0, 60.0]
CRS_0 = 90 # target's course
HDG_0 = 90 # o/s heading
SPD_0 = 1 # 1 or 2
TGT_SPD = 1

#input is course in degrees and rng
#returns next course in degrees
function next_crs(crs,rng)
    if rand(rng) < .9
        return crs
    end
    return (crs + rand(rng,[-1,1])*30 % 360)
end

# state as tuple (x, y, crs, hdg) of target (hdg of o/s)
function f(state, control, rng)
    r, θ, crs, hdg, spd = state
    x = r*cos(π/180*θ)
    y = r*sin(π/180*θ)
    pos = [x + TGT_SPD*cos(π/180*crs) - spd*cos(π/180*hdg), y + 
        TGT_SPD*sin(π/180*crs) - spd*sin(π/180*hdg)]
    crs = next_crs(crs,rng)
    hdg = hdg + control[1]
    hdg = hdg % 360
    if hdg < 0
        hdg += 360
    end
    spd = control[2]
    r = sqrt(pos[1]^2 + pos[2]^2)
    θ = trunc(Int, atan2(pos[1],pos[2])*180/π)
    if θ < 0 θ += 360 end
    return (r, θ, crs, hdg, spd)::NTuple{5, Real}
end


# likelihood weight of observation "o" given x, a, xp

x = []
courses = []
observs = []
crs = CRS_0
pos = POS_0
hdg = HDG_0
spd = SPD_0

for i in 1:100
    state = f((pos[1], pos[2], crs, hdg, spd),(0,1),rng)
    pos[1], pos[2], crs, hdg, spd = state
    θ = state[2]
    rad = state[1]
    if θ < 0 θ += 360 end
    push!(courses,crs)
    push!(observs,(rad, θ, obs0(state), obs1(state), obs2(state), obs3(state)))
end


In [ ]:

#for i in observs
    #println(i)
#end


In [40]:
function r(s::NTuple{5,Real})
    range = s[1]
    if range > 100 return -1 end  # reward to not lose track of contact
    if range <= 10 return -100 end  # collision avoidance
    return 1  # being in "sweet spot" maximizes reward
end

r (generic function with 1 method)

In [347]:
# implement POMDP here

mutable struct targetPOMDP <: POMDP{NTuple{5,Real}, Tuple{Float64}, Int64}
    p_walk::Float64
    discount::Float64
end
angles = [0, 30, 60, 90, 120, 150, 210, 240, 270, 300, 330]
#statespace = [(10.0*r,θ*30.0, crs*30.0, hdg*30.0, spd) for r in 0:30, θ in 0:11, crs in 1:12, hdg in 1:12, spd in 1:2]
statespace = thestates
actionspace = ((30,1), (0,1), (-30,1), (30, 2), (0,2), (-30,2))


targetPOMDP() = targetPOMDP(0.9, 0.9)
POMDPs.actions(::targetPOMDP) = ((30,1), (0,1), (-30,1), (30, 2), (0,2), (-30,2))
POMDPs.actionindex(::targetPOMDP, a) = (a[1]/30+1)*2 + a[2]
POMDPs.states(::targetPOMDP) = statespace
POMDPs.stateindex(::targetPOMDP, s::NTuple{5,Real}) = LinearIndices(statespace)[s[1]/10, s[2]/30, s[3]/30, s[4]/30, s[5]]
#POMDPs.stateindex(::targetPOMDP, s::NTuple{5,Int64}) = LinearIndices(statespace)[round(Int,s[1]/10)+1, round(Int,s[2]/30)+1,
#    (round(Int,s[3]/30)+1, round(Int,s[4]/30)+1, round(Int,s[5])+1]
POMDPs.observations(::targetPOMDP) = (0, 1, 2, 3)
POMDPs.obsindex(::targetPOMDP, o::Int64) = o + 1
POMDPs.initialobs(::targetPOMDP, s::NTuple{5,Real}, rng::AbstractRNG) = h(s,rng)
POMDPs.initialstate_distribution(::targetPOMDP) = ParticleCollection([(5, 60, 90, 90, 1) for i in 1:N])
POMDPs.initialstate(::targetPOMDP, rng::AbstractRNG) = (rand(rng,1:10), rand(rng, angles),
    rand(rng, angles), rand(rng, angles), rand(rng,1:2))
POMDPs.isterminal(::targetPOMDP, s) = s[1] >= 300
POMDPs.discount(::targetPOMDP) = 0.9

function POMDPs.gen(m::targetPOMDP, s, a, rng)
    return (sp=f(s,a,rng), r=r(s), o=h(s,rng))
end

In [122]:
N = 1000
pomdp = targetPOMDP(0.9, 0.9)
#initialstate_distribution(pomdp) = ParticleCollection([[5, 60, 90, 90, 1] for i in 1:N])


updater = SIRParticleFilter(pomdp, N);

function f2(x, u, rng)
    return [i for i in f(x, u, rng)]
end


model = ParticleFilterModel{Vector{Float64}}(f2, g)
pfilter = SIRParticleFilter(model, N);


In [349]:
actionspace[3]

(-30, 1)

In [353]:
θ = zeros(length(grid),6);
β = weighted_grid_2(b)
θ[:,3] += 5*β
θ[:,5] += 6*β
transpose(θ[:,3])*β
           
argmax2([transpose(θ[:,i])*β for i in 1:size(θ)[2]],rng)
    
                    

5

In [209]:
## particle filter

γ = 0.9
b = ParticleCollection([[15, 60, 90, 90, 1] for i in 1:N])
x = [15, 60, 90, 90, 1];
plots = []
total = 0
for i in 1:200
    print(".")
    if i % 10 == 0
        u = (30 * rand(rng,[1,-1]), rand(rng, [1,2]))
    else
        u = (0,1)
    end
    x = f2(x, u, rng)
    y = h(x, rng)
    b = update(pfilter, b, u, y)
    rew = r(Tuple(x))
    total += rew
    x, y, rew
    
    # plotting
    #r_ = [row[1] for row in particles(b)]
    #theta = [row[2] for row in particles(b)]*π/180
    #x_theta = x[2]*π/180
    #x_r = x[1]
      
    #plt = plot(proj=:polar, lims=(0,100), size=(1000,1000))
    #scatter!(theta, r_, markersize=1, label="particles")
    #scatter!([x_theta], [x_r], markersize=3, label="target")
    #push!(plots, plt)    
end
total

........................................................................................................................................................................................................

-1420

In [344]:

β = weighted_grid(b);

In [ ]:
env = POMDPEnvironment(targetPOMDP())


function simulate(env::AbstractEnvironment, nsteps::Int = 100)
    done = false
    r_tot = 0.0
    step = 1
    o = reset!(env)
    while !done && step <= nsteps
        action = sample_action(env) # take random action 
        action
        obs, rew, done, info = step!(env, action)
        obs = trunc(Int,obs[1])
        #@show obs, rew, done, info
        r_tot += rew
        step += 1
    end
    return r_tot
end

@show simulate(env)
#ac = sample_action(env)
#step!(env, ac)